In [40]:
# Import libraries
import glob
import pandas as pd
import dask.dataframe as dd

cases = pd.concat([pd.read_csv(f) for f in glob.glob('./csv/cases/*.csv') if int(f[18:22]) > 2015], ignore_index = True)

In [41]:
cases.head()

,ddl_case_id,year,state_code,dist_code,court_no,cino,judge_position,female_defendant,female_petitioner,female_adv_def,female_adv_pet,type_name,purpose_name,disp_name,date_of_filing,date_of_decision,date_first_list,date_last_list,date_next_list
0,01-01-01-201900000012016,2016,1,1,1,MHNB030000032016,chief judicial magistrate,0 male,0 male,-9999,0,1940.0,767.0,26,2016-01-02,2016-02-16,2016-01-02,2016-02-16,2016-02-16
1,01-01-01-201900000022016,2016,1,1,1,MHNB030000042016,chief judicial magistrate,0 male,0 male,-9999,0,1940.0,767.0,26,2016-01-02,2016-02-16,2016-01-02,2016-02-16,2016-02-16
2,01-01-01-201900000032016,2016,1,1,1,MHNB030000052016,chief judicial magistrate,0 male,1 female,-9999,0,1940.0,4878.0,43,2016-01-02,2016-02-13,2016-01-02,2016-02-13,2016-02-13
3,01-01-01-201900000042016,2016,1,1,1,MHNB030000062016,chief judicial magistrate,0 male,0 male,-9999,1,1940.0,7430.0,23,2016-01-05,2017-06-07,2016-01-05,2017-06-07,2017-06-07
4,01-01-01-201900000052016,2016,1,1,1,MHNB030000072016,chief judicial magistrate,0 male,0 male,-9999,0,1940.0,5251.0,26,2016-01-06,2016-02-18,2016-01-06,2016-02-18,2016-02-18


In [45]:
cases_pre = cases    
cases_pre = cases_pre.drop(['date_first_list', 'date_last_list', 'date_next_list', 'cino'], axis=1)

In [28]:
cases_pre.head()

,ddl_case_id,year,state_code,dist_code,court_no,judge_position,female_defendant,female_petitioner,female_adv_def,female_adv_pet,type_name,purpose_name,disp_name,date_of_filing,date_of_decision
0,01-01-01-201900000012016,2016.0,1.0,1.0,1.0,chief judicial magistrate,0 male,0 male,-9999.0,0.0,1940.0,767.0,26.0,2016-01-02,2016-02-16
1,01-01-01-201900000022016,2016.0,1.0,1.0,1.0,chief judicial magistrate,0 male,0 male,-9999.0,0.0,1940.0,767.0,26.0,2016-01-02,2016-02-16
2,01-01-01-201900000032016,2016.0,1.0,1.0,1.0,chief judicial magistrate,0 male,1 female,-9999.0,0.0,1940.0,4878.0,43.0,2016-01-02,2016-02-13
3,01-01-01-201900000042016,2016.0,1.0,1.0,1.0,chief judicial magistrate,0 male,0 male,-9999.0,1.0,1940.0,7430.0,23.0,2016-01-05,2017-06-07
4,01-01-01-201900000052016,2016.0,1.0,1.0,1.0,chief judicial magistrate,0 male,0 male,-9999.0,0.0,1940.0,5251.0,26.0,2016-01-06,2016-02-18


In [48]:
print(cases_pre.shape)

(38139072, 15)


In [50]:
# find number of unique judge_positions values
print(cases_pre.judge_position.nunique())

521


In [51]:
cases_pre['judge_position'] = cases_pre['judge_position'].astype('category')

In [52]:
cases_pre.memory_usage(deep=True)

Index                       128
ddl_case_id          3089264832
year                  305112576
state_code            305112576
dist_code             305112576
court_no              305112576
judge_position         76337880
female_defendant     2472063909
female_petitioner    2553190574
female_adv_def        305112576
female_adv_pet        305112576
type_name             305112576
purpose_name          305112576
disp_name             305112576
date_of_filing       2555317824
date_of_decision     2061557774
dtype: int64

In [54]:
cases_pre = cases_pre.drop_duplicates()

In [56]:
# downcast the integer and float columns

cases_pre[['year', 'state_code', 'dist_code', 'court_no', 'disp_name']] = cases_pre[['year', 'state_code', 'dist_code', 'court_no', 'disp_name']].apply(pd.to_numeric, downcast='unsigned')
cases_pre[['type_name', 'purpose_name']] = cases_pre[['type_name', 'purpose_name']].apply(pd.to_numeric, downcast='float')


In [57]:
cases_pre.memory_usage(deep=True)

Index                 305112576
ddl_case_id          3089264832
year                   76278144
state_code             38139072
dist_code              38139072
court_no               38139072
judge_position         76337880
female_defendant     2472063909
female_petitioner    2553190574
female_adv_def        305112576
female_adv_pet        305112576
type_name             152556288
purpose_name          152556288
disp_name              38139072
date_of_filing       2555317824
date_of_decision     2061557774
dtype: int64

In [59]:
cases_pre = cases_pre.drop(['date_of_filing', 'date_of_decision'], axis=1)

In [68]:
cases_pre['female_defendant'] = cases_pre['female_defendant'].str.replace('[a-zA-Z]', '').astype('int')
cases_pre['female_petitioner'] = cases_pre['female_petitioner'].str.replace('[a-zA-Z]', '').astype('int')

In [66]:
cases_pre.head()

,ddl_case_id,year,state_code,dist_code,court_no,judge_position,female_defendant,female_petitioner,female_adv_def,female_adv_pet,type_name,purpose_name,disp_name
0,01-01-01-201900000012016,2016,1,1,1,chief judicial magistrate,0,0,-9999,0,1940.0,767.0,26
1,01-01-01-201900000022016,2016,1,1,1,chief judicial magistrate,0,0,-9999,0,1940.0,767.0,26
2,01-01-01-201900000032016,2016,1,1,1,chief judicial magistrate,0,1,-9999,0,1940.0,4878.0,43
3,01-01-01-201900000042016,2016,1,1,1,chief judicial magistrate,0,0,-9999,1,1940.0,7430.0,23
4,01-01-01-201900000052016,2016,1,1,1,chief judicial magistrate,0,0,-9999,0,1940.0,5251.0,26


In [71]:
# remove all rows with missing values
cases_pre = cases_pre.dropna()

# if the value of female_petitioner is -9999, remove the row
cases_pre = cases_pre.loc[(cases_pre['female_adv_def'] != -9999) & (cases_pre['female_adv_def'] != -9998)]
cases_pre = cases_pre.loc[(cases_pre['female_adv_pet'] != -9999) & (cases_pre['female_adv_pet'] != -9998)]
cases_pre = cases_pre.loc[(cases_pre['female_defendant'] != -9999) & (cases_pre['female_defendant'] != -9998)]
cases_pre = cases_pre.loc[(cases_pre['female_petitioner'] != -9999) & (cases_pre['female_petitioner'] != -9998)]


In [72]:
cases_pre.shape

(637512, 13)

In [37]:
cases_pre = cases
acts_sections_pre = acts_sections
judges_pre = judges

NameError: name 'acts_sections' is not defined

In [8]:
acts_sections.head()

AttributeError: 'TextFileReader' object has no attribute 'head'